Import all modules the will be used and set up date that will be used as input for the TD Ameritrade API according to the standards set out in the API Documentation: https://developer.tdameritrade.com/option-chains/apis/get/marketdata/chains

In [1]:
import time
import datetime
import requests
import pandas
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

API_KEY = "***INSERT API KEY***"

STOCKS_TO_SCREEN = ["QQQ"]
YEAR = "2021"
MONTH = "09"
DAY = "08"
DAYS_TO_EXP = "1"

#fill in all the dates that you would like to collect and should apply to all options
API_DATE_INPUT = "{}-{}-{}:{}".format(YEAR,MONTH,DAY,DAYS_TO_EXP)

Collect all the data that will be collected and used in the dashboard

In [2]:
def collect_option_data(symbol,strike,contractType="CALL"):
    #Collect all relevant information about an option that will be analyzed
    parameters = {"apikey":API_KEY,
                 "includeQuotes":True,
                 "strike":str(strike),
                 "symbol":symbol,
                 "contractType":contractType}
        
    url = "https://api.tdameritrade.com/v1/marketdata/chains"
    response = requests.get(url,params=parameters).json()
    pointer = response['callExpDateMap'][API_DATE_INPUT][str(parameters['strike'])][0]
    
    data = {"bidPrice":0,"bidSize":0,"askPrice":0,"askSize":0,"totalVolume":0,"volatility": 0,
            "delta": 0,"gamma": 0,"theta": 0,"vega": 0,"rho": 0,}
    
    data["askPrice"] = pointer["ask"] 
    data["askSize"] = pointer["askSize"] 
    data["bidPrice"] = pointer["bid"] 
    data["bidSize"] = pointer["bidSize"]
    data["totalVolume"] = pointer["totalVolume"]
    data["volatility"] = pointer["volatility"]
    data["delta"] = pointer["delta"]
    data["gamma"] = pointer["gamma"]
    data["theta"] = pointer["theta"]
    data["vega"] = pointer["vega"]
    data["rho"] = pointer["rho"]
    
    return data
    
print(collect_option_data("QQQ","381.0"))

{'bidPrice': 1.58, 'bidSize': 10, 'askPrice': 1.62, 'askSize': 20, 'totalVolume': 464, 'volatility': 13.086, 'delta': 0.559, 'gamma': 0.117, 'theta': -0.396, 'vega': 0.102, 'rho': 0.009}


In [3]:
PRICE_DATA = {}

def create_hashmap_price(min_price,max_price,step=0.01):
    #creates a hashmap that will store the prices and the current bid and asks
    global PRICE_DATA
    for current_price in range(int(min_price*100),int((step+max_price)*100),int(step*100)):
        current_price_string = str(current_price/100)
        PRICE_DATA[float(current_price_string)] = {"bid":0,"ask":0}
        
def update_data_bidask(given_update):
    #undates the orderbook data to reflect current prices
    global PRICE_DATA
    for current_price in PRICE_DATA:
        if current_price > given_update["bidPrice"]:
            PRICE_DATA[current_price]["bid"] = 0
        elif current_price == given_update["bidPrice"]:
            PRICE_DATA[current_price]["bid"] = given_update["bidSize"]
            
        if current_price < given_update["askPrice"]:
            PRICE_DATA[current_price]["ask"] = 0
        elif current_price == given_update["askPrice"]:
            PRICE_DATA[current_price]["ask"] = given_update["askSize"]
    return None

def create_ask_graph_data(given_price_data,cumulative=True):
    #creates the data that will be graphed in later functions
    ask_data = {}
    total_asks = 0
    for current_price in PRICE_DATA:
        total_asks += given_price_data[current_price]["ask"]
        if total_asks != 0:
            if cumulative:
                ask_data[current_price] = total_asks
            else:
                ask_data[current_price] = given_price_data[current_price]["ask"]
                
    return ask_data

def create_bid_graph_data(given_price_data,cumulative=True):
    #creates the data that will be graphed in later functions
    bid_data = {}
    total_bids = 0
    for current_price,current_bidask_dict in sorted(list(given_price_data.items()), key=lambda x:x, reverse=True):
        total_bids += given_price_data[current_price]["bid"]
        if total_bids != 0:
            if cumulative:
                bid_data[current_price] = total_bids
            else:
                bid_data[current_price] = given_price_data[current_price]["bid"]
    return bid_data

Creates a hashmap that will contain all the data we will be graphing

In [26]:
min_price,max_price = 1.0,3.0
create_hashmap_price(min_price,max_price)

All the graphing functions that we will need for the dashboard

In [ ]:
def create_orderbook_chart(given_price_data):
    bid_data = create_bid_graph_data(given_price_data,False)
    ask_data = create_ask_graph_data(given_price_data,False)
    
    fig = go.Figure(data=[
    go.Bar(name='bids',y=list(bid_data.values()),x=list(bid_data.keys()),opacity=0.8,marker_color='rgb(93, 245, 66)'),
    go.Bar(name='asks',y=list(ask_data.values()),x=list(ask_data.keys()),opacity=0.8,marker_color='rgb(247, 63, 17)')
])
    # Change the bar mode
    fig.update_layout(barmode='group',bargap=0)
    
    fig.show()

def create_depth_chart(given_price_data):
    bid_data = create_bid_graph_data(given_price_data,True)
    ask_data = create_ask_graph_data(given_price_data,True)
    fig = go.Figure(data=[
    go.Bar(name='bids', x=list(bid_data.keys()), y=list(bid_data.values()),opacity=0.8,marker_color='rgb(93, 245, 66)'),
    go.Bar(name='asks', x=list(ask_data.keys()), y=list(ask_data.values()),opacity=0.8,marker_color='rgb(247, 63, 17)')
])
    # Change the bar mode
    fig.update_layout(barmode='group',bargap=0)
    
    fig.show()
    
def create_greek_chart(desired_greek):
    #graph the desired Greek from the file data or live updates
    dataframe = pandas.read_csv("{}.csv".format(desired_greek))
    xaxis_title="Time"
    yaxis_title="{} Graph".format(desired_greek),
    
    fig = go.Figure(data=go.Ohlc(x=df['Time'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close']))
    
    fig.update(layout_xaxis_rangeslider_visible=False)
    
    fig.update_layout(title="{}".format(file_location.split(".")[0]))
    fig.show()
    
def create_volume_chart(file_name="Volume.csv"):
    running_total = 0
    dataframe = pandas.read_csv(file_name)
    dataframe.to_dict()
    
    volume_data = {}
    running_total = 0
    for current_row in dataframe["Time"]:
        current_time = dataframe["Time"][current_row]
        current_volume = dataframe["Volume"][current_row]
        running_total += current_volume
        volume_data[current_time] = running_total
    
    fig = go.Figure(data=[
    go.Bar(name='Volume',x=list(volume_data.keys()),y=list(volume_data.values()),opacity=0.8,marker_color='rgb(245,188,17)')
])
    # Change the bar mode
    fig.update_layout(barmode='group',bargap=0.3)
    
    fig.show()
    

Run and update the graphs througout the day to see market updates

In [32]:
count = 0
while count < 50:
    current_information = collect_option_data("QQQ","381.0")
    #update the order book
    update_data_bidask(current_information)
    #create the bid ask spread
    ask_data = create_ask_graph_data(PRICE_DATA)
    bid_data = create_bid_graph_data(PRICE_DATA)
    #grapth the bid ask data
    print (count)
    create_depth_chart(PRICE_DATA)
    create_orderbook_chart(PRICE_DATA)
    time.sleep(5)
    count += 1

0


1


2


3


4


...

45


46


47


48


49


The code below is used to ccreate files to store data and then to write that data to file

In [6]:
import csv

greeks = ["Theta","Delta","Vega","Rho","Gamma"]
# open the file in the write mode
for current_greek in greeks:
    f = open('{}.csv'.format(current_greek), 'w')
    row = ["Time","Open","High","Low","Close"]
    writer = csv.writer(f)
    writer.writerow(row)
    # close the file
    f.close()
    
def write_data_to_file(file_name,data_array):
    data_line_to_write = "\n"
    for current_value in data_array:
        data_line_to_write += str(current_value)
    
    with open(file_name,"a") as file_data:
        file_data.write(data_line_to_write)

Update the Greek files using the API calls

In [ ]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M")
print("Current Time =", current_time)

greeks = ["Theta","Delta","Vega","Rho","Gamma","Volatility"]
GREEK_DATA = {}

#create the greek hashmap that will collect data
update = collect_option_data("QQQ","381.0")
for current_greek_cap in greeks:
    current_greek = current_greek_cap.lower()
    GREEK_DATA[current_greek] = {"Open":update[current_greek],
                                 "High":update[current_greek],
                                 "Low":update[current_greek],
                                 "Close":update[current_greek],}
    
current_time = now.strftime("%H:%M")
while current_time != "09:30":
    current_time = now.strftime("%H:%M")
    current_time_long = now.strftime("%H:%M:%S")
    print (current_time_long)
    
starting_time = now.strftime("%H:%M")

while True:
    update = collect_option_data("QQQ","381.0")
    
    now = datetime.now()
    
    current_time = now.strftime("%H:%M")
    current_time_long = now.strftime("%H:%M:%S")
    if starting_time != current_time:
        print ("WRITING INFORMATION")
        for current_greek in GREEK_DATA:
            data_to_add = [starting_time]
            write_data_to_file("Volume.csv",update["totalVolume"])
            for current_data_point in GREEK_DATA[current_greek]:
                data_to_add.append(str(current_data_point))
                write_data_to_file("{}.csv".format(current_greek),data_to_add)
                #reset the hashmap
                GREEKS[current_greek]["Open"] = update[current_greek]
                GREEKS[current_greek]["High"] = update[current_greek]
                GREEKS[current_greek]["Low"] = update[current_greek]
                GREEKS[current_greek]["Close"] = update[current_greek]
            #reset the starting time
            starting_time = current_time
    else:
        print ("UPDATING INFOMRATION AT: {}".format(current_time_long))
        for current_greek in GREEK_DATA:
            current_greek = current_greek.lower()
            GREEK_DATA[current_greek]["High"] = max(GREEK_DATA[current_greek]["High"],update[current_greek])
            GREEK_DATA[current_greek]["Low"] = min(GREEK_DATA[current_greek]["Low"],update[current_greek])
            GREEK_DATA[current_greek]["Close"] = update[current_greek]

    time.sleep(10)